In [20]:
import sys
sys.path.append("../_build/")

import constants

In [24]:
df = constants.read_poi_file()    
    
completed = constants.get_completed(df)[[constants.DATE_COL, constants.TO_SPRINGER_COL]]
completed[constants.DATE_COL+"_dt"] = completed[constants.DATE_COL].dt.strftime('%Y-%m-%d')

print(len(completed))
print(len(df))
completed.head(10)

170
546


,dt_reached,to_spgr,dt_reached_dt
0,2017-03-13 10:03:00,-8.8,2017-03-13
1,2017-03-13 13:37:00,-1.5,2017-03-13
2,2017-03-14 11:37:00,-1.5,2017-03-14
3,2017-03-14 12:14:00,0.0,2017-03-14
4,2017-03-14 12:25:00,0.2,2017-03-14
5,2017-03-14 12:47:00,1.0,2017-03-14
6,2017-03-14 13:18:00,2.0,2017-03-14
7,2017-03-14 13:42:00,2.8,2017-03-14
8,2017-03-14 15:02:00,4.3,2017-03-14
9,2017-03-15 12:02:00,4.3,2017-03-15


In [25]:
# Need to shift the values, since the last checkpoint of each day should be the first checkpoint of the next day
completed[constants.TO_SPRINGER_COL + '_shifted'] = completed[constants.TO_SPRINGER_COL].shift(1)
completed[constants.DATE_COL + '_shifted'] = completed[constants.DATE_COL].shift(1)
completed[constants.DATE_COL + '_dt_shifted'] = completed[constants.DATE_COL + '_dt'].shift(1)

completed.head(10)

,dt_reached,to_spgr,dt_reached_dt,to_spgr_shifted,dt_reached_shifted,dt_reached_dt_shifted
0,2017-03-13 10:03:00,-8.8,2017-03-13,NaN,NaT,NaN
1,2017-03-13 13:37:00,-1.5,2017-03-13,-8.8,2017-03-13 10:03:00,2017-03-13
2,2017-03-14 11:37:00,-1.5,2017-03-14,-1.5,2017-03-13 13:37:00,2017-03-13
3,2017-03-14 12:14:00,0.0,2017-03-14,-1.5,2017-03-14 11:37:00,2017-03-14
4,2017-03-14 12:25:00,0.2,2017-03-14,0.0,2017-03-14 12:14:00,2017-03-14
5,2017-03-14 12:47:00,1.0,2017-03-14,0.2,2017-03-14 12:25:00,2017-03-14
6,2017-03-14 13:18:00,2.0,2017-03-14,1.0,2017-03-14 12:47:00,2017-03-14
7,2017-03-14 13:42:00,2.8,2017-03-14,2.0,2017-03-14 13:18:00,2017-03-14
8,2017-03-14 15:02:00,4.3,2017-03-14,2.8,2017-03-14 13:42:00,2017-03-14
9,2017-03-15 12:02:00,4.3,2017-03-15,4.3,2017-03-14 15:02:00,2017-03-14


In [70]:
# Exclude calculating the overnight duration by filtering where the dates are different
completed = completed[completed[constants.DATE_COL + '_dt'] == completed[constants.DATE_COL + '_dt_shifted']]

# Group by day
f = {constants.TO_SPRINGER_COL: 'last', 'to_spgr_shifted': 'first', constants.DATE_COL: 'last', constants.DATE_COL + '_shifted': 'first'}
miles_per_day = completed.groupby(constants.DATE_COL+'_dt').agg(f)
miles_per_day['miles'] = miles_per_day[constants.TO_SPRINGER_COL] - miles_per_day[constants.TO_SPRINGER_COL + '_shifted']
miles_per_day['duration'] = miles_per_day[constants.DATE_COL] - miles_per_day[constants.DATE_COL + '_shifted']

miles_per_day.head(10)

,to_spgr,to_spgr_shifted,dt_reached,dt_reached_shifted,miles,duration
dt_reached_dt,,,,,,
2017-03-13,-1.5,-8.8,2017-03-13 13:37:00,2017-03-13 10:03:00,7.3,03:34:00
2017-03-14,4.3,-1.5,2017-03-14 15:02:00,2017-03-14 11:37:00,5.8,03:25:00
2017-03-15,8.1,4.3,2017-03-15 13:46:00,2017-03-15 12:02:00,3.8,01:44:00
2017-03-16,15.8,8.1,2017-03-16 13:58:00,2017-03-16 08:56:00,7.7,05:02:00
2017-03-17,24.2,15.8,2017-03-17 14:34:00,2017-03-17 09:09:00,8.4,05:25:00
2017-03-18,31.4,24.2,2017-03-18 11:35:00,2017-03-18 07:16:00,7.2,04:19:00
2017-03-19,42.9,31.4,2017-03-19 12:54:00,2017-03-19 07:29:00,11.5,05:25:00
2017-03-20,52.6,42.9,2017-03-20 11:58:00,2017-03-20 07:58:00,9.7,04:00:00
2017-03-22,63.0,52.6,2017-03-22 14:02:00,2017-03-22 09:06:00,10.4,04:56:00


In [71]:
# Fix rounding issue by converting each mileage value into a string
def format_duration(d):
    seconds = d.total_seconds()
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    
    return '%02d:%02d' % (hours, minutes)

miles_per_day['miles'] = miles_per_day['miles'].apply(lambda x: '{:.1f}'.format(x))
miles_per_day['duration'] = miles_per_day['duration'].apply(lambda x: format_duration(x))

{'mileage': miles_per_day[['miles', 'duration']].T.to_dict()}

{'mileage': {'2017-03-13': {'duration': '03:34', 'miles': '7.3'},
  '2017-03-14': {'duration': '03:25', 'miles': '5.8'},
  '2017-03-15': {'duration': '01:44', 'miles': '3.8'},
  '2017-03-16': {'duration': '05:02', 'miles': '7.7'},
  '2017-03-17': {'duration': '05:25', 'miles': '8.4'},
  '2017-03-18': {'duration': '04:19', 'miles': '7.2'},
  '2017-03-19': {'duration': '05:25', 'miles': '11.5'},
  '2017-03-20': {'duration': '04:00', 'miles': '9.7'},
  '2017-03-22': {'duration': '04:56', 'miles': '10.4'},
  '2017-03-23': {'duration': '06:27', 'miles': '10.8'},
  '2017-03-24': {'duration': '06:25', 'miles': '12.2'},
  '2017-03-25': {'duration': '07:23', 'miles': '16.3'},
  '2017-03-26': {'duration': '02:39', 'miles': '7.2'},
  '2017-03-28': {'duration': '07:40', 'miles': '15.8'},
  '2017-03-29': {'duration': '06:21', 'miles': '11.7'},
  '2017-03-30': {'duration': '03:03', 'miles': '6.7'},
  '2017-03-31': {'duration': '04:15', 'miles': '9.1'},
  '2017-04-01': {'duration': '05:14', 'miles': 

In [ ]:
mpd = {'miles_per_day': miles_per_day['miles'].to_dict()}

with open(constants.statsFilePath, 'w') as outfile:
    json.dump({**mpd}, outfile)